In [ ]:
!pip install azure-ai-formrecognizer --pre

In [64]:
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__


In [66]:
endpoint = "<your endpoint>"
key = "<your key>"

In [67]:
form_recognizer_client = FormRecognizerClient(endpoint, AzureKeyCredential(key))

In [99]:
def print_result(invoices, blob_name):
    for idx, invoice in enumerate(invoices):
        print("--------Recognizing invoice {}--------".format(blob_name))
        vendor_name = invoice.fields.get("VendorName")
        if vendor_name:
            print("Vendor Name: {} has confidence: {}".format(vendor_name.value, vendor_name.confidence))
        vendor_address = invoice.fields.get("VendorAddress")
        if vendor_address:
            print("Vendor Address: {} has confidence: {}".format(vendor_address.value, vendor_address.confidence))
        customer_name = invoice.fields.get("CustomerName")
        if customer_name:
            print("Customer Name: {} has confidence: {}".format(customer_name.value, customer_name.confidence))
        customer_address = invoice.fields.get("CustomerAddress")
        if customer_address:
            print("Customer Address: {} has confidence: {}".format(customer_address.value, customer_address.confidence))
        customer_address_recipient = invoice.fields.get("CustomerAddressRecipient")
        if customer_address_recipient:
            print("Customer Address Recipient: {} has confidence: {}".format(customer_address_recipient.value, customer_address_recipient.confidence))
        invoice_id = invoice.fields.get("InvoiceId")
        if invoice_id:
            print("Invoice Id: {} has confidence: {}".format(invoice_id.value, invoice_id.confidence))
        invoice_date = invoice.fields.get("InvoiceDate")
        if invoice_date:
            print("Invoice Date: {} has confidence: {}".format(invoice_date.value, invoice_date.confidence))
        invoice_total = invoice.fields.get("InvoiceTotal")
        if invoice_total:
            print("Invoice Total: {} has confidence: {}".format(invoice_total.value, invoice_total.confidence))
        due_date = invoice.fields.get("DueDate")
        if due_date:
            print("Due Date: {} has confidence: {}".format(due_date.value, due_date.confidence))


In [102]:
# Create the BlobServiceClient object which will be used to get the container_client
connect_str = "<your connection string>"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
raw_container_client = blob_service_client.get_container_client("raw")
processed_container_client = blob_service_client.get_container_client("processed")
invoiceUrlBase = raw_container_client.primary_endpoint
print(invoiceUrlBase)

https://storageaccountaiaprab2d.blob.core.windows.net/raw


In [109]:
print("\nProcessing blobs...")

# List the blobs in the container
blob_list = raw_container_client.list_blobs()
for blob in blob_list:
    invoiceUrl = f'{invoiceUrlBase}/{blob.name}'
    print(invoiceUrl)
    poller = form_recognizer_client.begin_recognize_invoices_from_url(invoiceUrl)
    # Get results
    invoices = poller.result()
    # Print results
    print_result(invoices, blob.name)
    # Copy blob to processed
    processed_container_client.upload_blob(blob, blob.blob_type,    overwrite=True)
    # Delete blob from raw now that its processed
    raw_container_client.delete_blob(blob)


Processing blobs...
https://storageaccountaiaprab2d.blob.core.windows.net/raw/Invoice103563.pdf
--------Recognizing invoice Invoice103563.pdf--------
Vendor Name: T.T TAILWIND TRADERS has confidence: 0.992
Vendor Address: 488 Meadow Glen Way Lincoln Acres has confidence: 0.377
Customer Name: Brian Groth has confidence: 0.25
Customer Address: 1145 Paraiso Ct. has confidence: 0.932
Customer Address Recipient: Brian Groth has confidence: 0.25
Invoice Id: 103563 has confidence: 0.994
Invoice Total: 513.6 has confidence: 0.996
https://storageaccountaiaprab2d.blob.core.windows.net/raw/Invoice106511.pdf
--------Recognizing invoice Invoice106511.pdf--------
Vendor Name: T.T TAILWIND TRADERS has confidence: 0.992
Customer Name: Wfownosec has confidence: 0.692
Customer Address: 6206 Arthur Rd has confidence: 0.92
Customer Address Recipient: Wfownosec has confidence: 0.692
Invoice Id: 106511 has confidence: 0.996
Invoice Total: 873.12 has confidence: 0.995
https://storageaccountaiaprab2d.blob.co